# Flux toward a tunnel from a large cavity 

August 2020

In [2]:
import numpy as np
from scipy.optimize import fsolve

In [7]:
g = 9.81 # Acceleration of gravity [m/s2]

def f_karman(Rr):
    """ 
    Computes the friction factor for a given rugosity assuming that 
    the Reynolds number is very large
    """
    return 1/(2*np.log10(Rr/3.7))**2

def flux_from_cavity(h,d,L,Rr,verbose=False,rho=998,mu=1.3e-3, cs = 0.5):
    """ 
    Computes the flux from a cavity through a karst conduit to a tunnel 
    using Bernoulli and Darcy-Weisbach equations
    
    input:
    
     h = height of the water level in cavity above the tunnel in [m]
     d = diameter of the conduits in [m]
     L = length of the horizontal condtuit in [m]
     Rr = rugosity of the conduits [unitless ratio] (should be < 0.1)
     
     verbose = a boolean flag, if equals True it prints intermediate results
     rho, mu = density and viscosity
     cs = singular head losses (by default only entrance into conduit)
    
    output:
    
     q = flux in [m3/s]
     
    method:
     a non linear equation solver is used 
     
     two coupled equations are considered
     - one coming from Bernoulli energy balance and Darcy Weisbach
     - the Colenbrook equation to determine the friction factor 
     
     the non linear solver needs an initial solution, therefore solving 
     the problems requires two steps:
      1/ we get an approximate solution assuming that reynolds numbers is very high
      2/ we use this as a starting point for the non linear solver
    
    """    
    
    # Compute approximate solution
    f = f_karman(Rr) # friction factor for high Reynolds number (Karman equation)
    v = np.sqrt( 2*g*h / (1 + cs + f*L/d) ) # Solving Bernoulli for velocity
    
    # Define the system of non linear equations to solve
    def flowproblem(x):
        """
        Takes as an input the two unknowns:
         x[0] is the velocity, and x[1] the friction factor
         
        Returns the values computed for the two equations to solve. These values
        should be equal to zero.
        """
        
        # This is just to improve readibility 
        v,f = x[0], x[1] # flow velocity [m/s] and friction factor
        
        # Initialize the values returned by the function
        y = np.zeros(2)

        # Bernoulli equation: y[0] should be equal to 0 to ensure energy balance
        y[0] = v**2 * (1+ cs + f*L/d)  - 2*g*h
        
        # Colebrook equation written as a function of f and v
        # The Reynolds number is written explicitly
        y[1] = 1/np.sqrt(f) + 2*np.log10(Rr/3.7 + 2.51*mu/(v*rho*d*np.sqrt(f)))
 
        return y
    
    # Solve the non linear problem
    x = fsolve( flowproblem, [v,f] )
    v,f = x[0], x[1]

    # Compute the flux in [m3/s]
    q = np.pi * (d/2)**2 * v
    
    # To print detailed results
    if verbose:
        y = flowproblem(x) # For convergence check
        print('input:')
        print('------')
        print(' total height:',h,'[m]')
        print(' diameter:',d,'[m]')
        print(' length:',L,'[m]')
        print(' rugosity:',Rr,'[unitless]')
        print(' mu/rho:',mu/rho,'[m2*s]')
        print(' cs:',cs,'[-]')
        print('output:')
        print('-------')
        print(' velocity:',v,'[m/s]')
        print(' flux:',q,'[m3/s]')
        print(' friction factor:',f,'[unitless]')
        print(' Reynolds number:',v*rho*d/mu,'[unitless]')
        print('convergence check:')
        print('-------')
        print(' Bernoulli equation:',y[0])
        print(' Colenbrook equation:',y[1])
    
    return q


# Testing the function

Comparison with the case presented in the course of C. Ancey (page 203)

According to C. Ancey, with $\rho=10^3$ and $\mu=10^{-3}$, and $cs=1.8$ we should get:

- $v = 5.86$
- $f = 0.00863$
- $Re = 5.85811e6$

The results are not too far. Orders of magnitude are respected but not exactly identical. I do not find the reason for the slight difference. It should be related to the 

In [6]:
h = 20
d = 1
L = 1000
Rr = 1e-5

q = flux_from_cavity(h, d, L, Rr, verbose=True, rho=1e3, mu=1e-3 , cs = 1.8) 

input:
------
 total height: 20 [m]
 diameter: 1 [m]
 length: 1000 [m]
 rugosity: 1e-05 [unitless]
 mu/rho: 1e-06 [m2*s]
 cs: 1.8 [-]
output:
-------
 velocity: 5.6558245206816755 [m/s]
 flux: 4.442074191041641 [m3/s]
 friction factor: 0.009466965555452794 [unitless]
 Reynolds number: 5655824.5206816755 [unitless]
convergence check:
-------
 Bernoulli equation: -2.5579538487363607e-12
 Colenbrook equation: 2.078337502098293e-13


In [8]:
'''
input:

 h = height of the water level in cavity above the tunnel in [m]
 d = diameter of the conduits in [m]
 L = length of the horizontal condtuit in [m]
 Rr = rugosity of the conduits [unitless ratio] (should be < 0.1)

 verbose = a boolean flag, if equals True it prints intermediate results
 rho, mu = density and viscosity
 cs = singular head losses (by default only entrance into conduit)

output:

 q = flux in [m3/s]
'''

# Now a test with parameters closer to our problem
d = 1
L = 5
Rr = 0.1
h = 100

q = flux_from_cavity(h, d, L, Rr, verbose=True,rho=1000) 

31.25623072338964 0.1016574165920041
input:
------
 total height: 100 [m]
 diameter: 1 [m]
 length: 5 [m]
 rugosity: 0.1 [unitless]
 mu/rho: 1.2999999999999998e-06 [m2*s]
 cs: 0.5 [-]
output:
-------
 velocity: 31.25623072338964 [m/s]
 flux: 24.548586204877118 [m3/s]
 friction factor: 0.1016574165920041 [unitless]
 Reynolds number: 24043254.402607415 [unitless]
convergence check:
-------
 Bernoulli equation: 0.0
 Colenbrook equation: 0.0


# To do 
Monte-Carlo simulations